In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
test_data = pd.read_csv('dota2Test.csv', header = None)
train_data = pd.read_csv('dota2Train.csv',header = None)

Data Cleaning
The first column is the result of game, -1 and 1 indicate which team win the match.
The rest of columns indicate the selection of heroes. One column is one hero. -1 and 1 indicate which team select the hero respectively and 0 means the hero is not chosen during this match.

In [3]:
train_data = train_data.drop(columns = [1,2,3])

In [4]:
train_data.rename(columns={0: "result"})

,result,4,5,6,7,8,9,10,11,12,...,107,108,109,110,111,112,113,114,115,116
0,-1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,-1,0,0,...,-1,0,0,0,0,0,0,0,0,0
4,-1,0,0,0,0,0,-1,0,0,-1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92645,-1,1,0,0,-1,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0
92646,1,0,0,0,0,-1,0,0,-1,0,...,1,0,0,0,0,0,0,0,0,0
92647,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
92648,-1,0,0,0,0,0,1,0,0,-1,...,0,0,0,0,0,0,0,0,0,0


In [5]:
test_data = test_data.drop(columns = [1,2,3])
test_data.rename(columns={0: "result"})

,result,4,5,6,7,8,9,10,11,12,...,107,108,109,110,111,112,113,114,115,116
0,-1,0,-1,0,0,0,0,0,0,1,...,-1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,...,-1,0,0,0,0,0,0,0,0,0
2,-1,1,0,0,0,-1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,1,-1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,-1,0,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10289,1,0,0,0,0,0,0,1,1,-1,...,0,-1,0,0,0,0,0,0,0,0
10290,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
10291,1,0,0,0,0,1,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
10292,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


PCA

In [6]:
x_train =train_data.iloc[:,1:]
x_test =test_data.iloc[:,1:]
y_train = train_data.iloc[:,0]
y_test = test_data.iloc[:,0]

In [7]:
pca = PCA(n_components = x_train.shape[1])
pca.fit(x_train)
x_train_pca = np.matmul(x_train, pca.components_.T).iloc[:,:70]

In [8]:
pca = PCA(n_components = x_test.shape[1])
pca.fit(x_test)
x_test_pca = np.matmul(x_test, pca.components_.T).iloc[:,:70]

In [9]:
x_train_pca

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.064393,0.109675,-1.053479,0.088103,-0.000588,0.066763,-0.785706,-0.504100,0.324986,0.894563,...,-0.061762,0.010197,-0.078898,0.007346,0.047479,0.021607,-0.015836,0.030289,0.008136,0.009908
1,0.033275,-0.082103,-1.134634,0.385266,0.664682,-0.064367,0.629649,-0.562047,-0.185120,-0.056284,...,0.807231,0.032471,0.056969,0.221501,0.081625,0.165953,-0.001365,-0.002956,-0.062498,0.062299
2,0.017679,-0.139369,-0.079958,0.064707,0.721789,-0.166121,0.447762,-1.444282,0.297247,0.154248,...,-0.129758,0.209531,-0.158642,-0.131494,0.406650,0.050315,-0.630100,-0.319007,-0.358241,0.096110
3,0.002062,-0.074709,-0.169431,-0.960091,-0.031529,-0.006573,-0.036267,-0.217946,-0.084985,-0.570762,...,-0.213570,0.227612,-0.126293,-0.037240,0.046240,0.016831,0.007491,0.024783,0.022393,-0.030088
4,1.129215,-0.889259,-0.115563,0.091284,0.093587,-0.700817,0.141380,-0.505482,-0.065273,-0.271262,...,-0.085585,-0.063053,0.062172,-0.127082,-0.101983,-0.189947,0.069204,-0.043870,0.038494,-0.122897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92645,0.910331,0.127414,-0.032710,-0.055068,0.056375,0.051167,-1.031246,-0.447939,-0.042748,-0.168519,...,-0.122886,0.045039,-0.016292,-0.094847,-0.078631,-0.017448,0.023947,-0.030255,-0.007784,-0.052289
92646,0.044963,-0.001825,0.228942,1.061078,-0.684102,0.481892,0.083729,-0.366988,-0.056721,0.028308,...,-0.081736,0.057691,-0.061621,-0.169635,-0.042604,-0.191616,-0.014823,-0.010553,0.096550,-0.126628
92647,-1.033410,-0.142177,-0.279280,0.025618,-0.028127,-0.129884,-0.973635,-0.843085,0.447148,1.002621,...,-0.163097,-0.151955,0.038567,-0.111559,-0.129189,-0.166405,0.048873,-0.090627,0.081595,-0.087218
92648,1.262216,-0.611101,-0.828650,0.176911,0.027336,0.713072,-0.379991,1.163967,-0.352726,-0.370561,...,-0.177166,0.319737,-0.511487,0.677637,-0.340963,0.008142,0.006272,0.007157,-0.061832,0.085200


Logistic Regression

In [10]:
clf = LogisticRegression(random_state=0).fit(x_train_pca, y_train)

In [11]:
clf.score(x_test_pca, y_test)

0.5210802409170391

In [12]:
clf.score(x_train_pca, y_train)

0.5879870480302213

KNN

In [13]:
knn = KNeighborsClassifier(n_neighbors = 2, weights = "distance")
knn.fit(x_train_pca,y_train)

KNeighborsClassifier(n_neighbors=2, weights='distance')

In [14]:
knn.score(x_test_pca, y_test)

0.5126287157567515

RandomForest

In [15]:
rf = RandomForestClassifier(max_depth = 2).fit(x_train_pca, y_train)

In [16]:
rf.score(x_test_pca, y_test)

0.5339032446085098

In [17]:
rf.score(x_train_pca, y_train)

0.5298758769562871